<a href="https://colab.research.google.com/github/Jaime44/WorkSpace/blob/main/models/content%20filter%20colaborative/content-based-recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive
    import tensorflow as tf
    print("Versión de TensorFlow:", tf.__version__)
    print("Dispositivos disponibles:", tf.config.list_physical_devices())

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/Models/content filter colaborative'

    path_workspace = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)

    # Verificar la GPU
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU no encontrada')
    print(f'Encontrada GPU: {device_name}')

    # Habilitar la GPU para TensorFlow
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memoria de la GPU configurada dinámicamente')
    else:
        print('No se encontraron dispositivos GPU configurables')
else:
    print("El código se está ejecutando en un entorno local.")
    path_workspace ='C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'
    path_absolute = os.getcwd().replace("\\", "/")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/Models/content filter colaborative'

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path


sys.path.append(path_workspace)

El código se está ejecutando en un entorno local.


![CBR](https://miro.medium.com/v2/resize:fit:1400/1*H_MMnrpLQrqTSJHdDOCMoA.png)

# What is Content Based Recommendation

Content-based recommendation, also known as content-based filtering, is a type of system or algorithm that provides recommendations to a user based on their interests and preferences. Those with such recommendation systems analyze the user's past preferences and likes, and suggest new items based on similar content.

Content-based recommendation analyzes the content of items and determines the ones that are suitable for the user based on similarity criteria. For example, when making a movie recommendation, the system can take into account the genres, actors, directors, and other features of the movies the user has liked or watched. Based on this information, the system suggests other movies with similar characteristics.

This recommendation system can utilize text analysis, tagging, categorization, or other content features along with the user profile or history to better understand the user's preferences. For instance, when making a music recommendation, the system can analyze features such as genres, instruments, tempo, and rhythm.

Content-based recommendation systems can be effective in providing personalized recommendations based on user preferences. The recommended items based on the user's past data can capture their interest and provide a better user experience.

# Business Problem
To recommend movies similar to the movies that a person who comes to our site to watch movies.

# Road Map

- 1. Creating the **TF-IDF Matrix**
- 2. Creation of **Cosine Similarity Matrix**
- 3. Making Recommendations Based on Similarities
- 4. Preparation of the Study Script


## Se cargan las librerías

In [2]:
# import Required Libraries

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Se cargan los Datasets

In [3]:
# df = pd.read_csv(path_absolute+"df_mezclado_tags_ratings_movies_links_genTags.csv")

In [4]:
# dataFrame_onehot_encode = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_onehot_encode.csv', sep=',')

# dataFrame_onehot_encode.head()


In [5]:
dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')
print(dataFrame_without_ohe.shape)
dataFrame_without_ohe.head()

C:\Users\jaime\AppData\Local\Temp\ipykernel_3432\4218327248.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')


(25000095, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,835444730,4.0


In [6]:
dataFrame = dataFrame_without_ohe.copy()


In [7]:
# dataFrame = dataFrame.sample(n=250000, random_state=42)
df = dataFrame.copy()
print(df.shape)
df.head()

(25000095, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,835444730,4.0


## Se eliminan las filas que tengan NaN en las columnas **tag_by_user** o **tag_genome**

In [8]:
def contar_valores_nulos(df):
    # Cuenta los valores nulos en cada columna del DataFrame
    valores_nulos_por_columna = df.isnull().sum()
    # Cuenta los valores nulos en todo el DataFrame
    total_valores_nulos = df.isnull().sum().sum()
    # Imprime la cantidad de valores nulos por columna
    print("Valores nulos por columna:")
    print(valores_nulos_por_columna)
    # Imprime el total de valores nulos en el DataFrame
    print("\nTotal de valores nulos en el dataset:", total_valores_nulos)
    
def eliminar_filas_nulas(df, columna):
    # Seleccionar las filas con valores nulos en la columna deseada
    filas_con_nulos = df[df[columna].isnull()]
    # Mostrar las filas con valores nulos
    # print("Filas con valores nulos en la columna", columna, ":")
    # print(filas_con_nulos)
    # Eliminar las filas con valores nulos en la columna deseada
    df = df.dropna(subset=[columna])
    return df

In [9]:
# contar_valores_nulos(df)
df = eliminar_filas_nulas(df, 'tag_genome')
# contar_valores_nulos(df)
df = eliminar_filas_nulas(df, 'tag_by_user')
# contar_valores_nulos(df)

In [10]:
print(df.shape)
df.head()

(197711, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
131,264,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1543393364,2.0
243,521,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1279793342,5.0
351,741,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1262230676,4.0
417,871,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1520214211,4.5
474,997,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1529249365,5.0


## Se toma un subDataset del original 

In [11]:
dataFrame = df.copy()
dataFrame = dataFrame.sample(n=20000, random_state=42)
df = dataFrame.copy()

In [12]:
# content_df = df[['title', 'genres', 'tag_by_user', 'tag_genome','rating']]
content_df = df[['title', 'genres', 'tag_by_user', 'tag_genome', 'timestamp','rating']]

In [13]:
print(content_df.shape)

(20000, 6)


## Funciones de limpieza de texto en Python

In [14]:
import re
import string

def remove_bars(text):
    """
    Elimina las barras verticales '|' del texto y las reemplaza por espacios.
    Parámetros:
    text (str): El texto de entrada que contiene barras verticales que se eliminarán.
    Retorna:
    str: El texto con las barras verticales eliminadas.
    """
    cleaned = re.sub(r'\|', ' ', text)
    return cleaned

def separate(text):
    """
    Divide el texto por comas, limpia cada parte eliminando el texto dentro de paréntesis, dígitos, espacios en blanco, puntuación, y lo convierte a minúsculas.
    Parámetros:
    text (str): El texto de entrada que contiene partes separadas por comas.
    Retorna:
    str: El texto limpiado y concatenado con cada parte separada por un espacio.
    """
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Elimina el texto dentro de paréntesis
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    """
    Elimina la puntuación y los dígitos del texto y lo convierte a minúsculas.
    Parámetros:
    text (str): El texto de entrada que se limpiará.
    Retorna:
    str: El texto limpiado con la puntuación y los dígitos eliminados, convertido a minúsculas.
    """
    try:
        cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text = cleaned.translate(str.maketrans('','', string.digits))
    except Exception as e:
        print(f"ERROR -----------------------------------------> {e} AND {text}")
    return clean_text


In [15]:
# Ejemplo de uso: Pixar Animation Studios Warner Bros., Lancaster Gate
# text_with_bars = "oldie but goodie"
# text_with_bars = "Pixar Animation Studios"
text_with_bars = "Warner Bros., Lancaster Gate""""  """
cleaned_text = remove_punc(text_with_bars)
print(cleaned_text)

warner bros lancaster gate  


## Se eliminan signos de puntuacin de genres, tag_by_user, tag_genome y barras de genres

In [16]:
content_df_aux = content_df.copy()
content_df_aux['genres'] = (content_df_aux['genres'].apply(remove_bars)).apply(remove_punc)
content_df_aux['tag_by_user'] = content_df_aux['tag_by_user'].apply(remove_punc)
content_df_aux['tag_genome'] = content_df_aux['tag_genome'].apply(remove_punc)
# content_df_aux.set_index('title', inplace=True)
content_df_aux = content_df_aux.reset_index()

In [17]:
content_df = content_df_aux.copy()
content_df.head()

,index,title,genres,tag_by_user,tag_genome,timestamp,rating
0,16524676,Eyes Wide Shut (1999),drama mystery thriller,mansion sexuality shocking scense toy store dv...,surreal enigmatic obsession sexual erotic,1432910964,4.5
1,20998795,Tokyo Story (Tôkyô monogatari) (1953),drama,sightsound gentle old man tea trip song chishu...,masterpiece criterion meditative tokyo japan,1458434226,5.0
2,23796499,Smokey and the Bandit (1977),action comedy,chase road trip police money trucking cb radio...,fun movie chase cars beer road movie,1424217798,4.0
3,24039979,Picnic at Hanging Rock (1975),drama mystery,lesbianism headmistress trip valentines day in...,surreal mystery australia enigmatic criterion,1241617829,5.0
4,20594541,"Jacket, The (2005)",drama mystery scifi thriller,experiment gfei own it john maybury love psych...,time travel psychological mindfuck amnesia future,1516077153,5.0


## Se crea la bag of words con las columnas genres, tag_by_user y tag_genome

In [18]:
df_aux = content_df.copy()
df_aux = df_aux[['genres', 'tag_by_user', 'tag_genome']]
df_aux['bag_of_words'] = ''
df_aux.loc[:, 'bag_of_words'] = df_aux.loc[:, df_aux.columns[0:]].apply(lambda x: ' '.join(x), axis=1)
# content_df.set_index('original_title', inplace=True)
columnas_a_eliminar = ['tag_by_user', 'tag_genome']
# Elimina las columnas especificadas del DataFrame 'data'
content_df = content_df.drop(columnas_a_eliminar, axis=1)

content_df['genres'] = df_aux['bag_of_words']
content_df.rename(columns={'genres': 'bag_of_words'}, inplace=True)

content_df.head()

,index,title,bag_of_words,timestamp,rating
0,16524676,Eyes Wide Shut (1999),drama mystery thriller mansion sexuality shock...,1432910964,4.5
1,20998795,Tokyo Story (Tôkyô monogatari) (1953),drama sightsound gentle old man tea trip song ...,1458434226,5.0
2,23796499,Smokey and the Bandit (1977),action comedy chase road trip police money tru...,1424217798,4.0
3,24039979,Picnic at Hanging Rock (1975),drama mystery lesbianism headmistress trip val...,1241617829,5.0
4,20594541,"Jacket, The (2005)",drama mystery scifi thriller experiment gfei o...,1516077153,5.0


## Creando la Matriz TF-IDF Matrix

La matriz TF-IDF es una herramienta esencial en sistemas de recomendación basados en contenido, ya que ayuda a calcular la similitud entre elementos de contenido utilizando su contenido textual, lo que mejora la precisión de las recomendaciones al encontrar elementos que comparten características semánticas.

In [19]:
df = content_df.copy()

In [20]:
df["bag_of_words"].head()

0    drama mystery thriller mansion sexuality shock...
1    drama sightsound gentle old man tea trip song ...
2    action comedy chase road trip police money tru...
3    drama mystery lesbianism headmistress trip val...
4    drama mystery scifi thriller experiment gfei o...
Name: bag_of_words, dtype: object

In [21]:
df["bag_of_words"].isnull().sum()

0

In [22]:
# Rellenemos el valor nulo de la variable cin bag_of_words con nada para evitar errores en los siguientes pasos
df['bag_of_words'] = df['bag_of_words'].fillna('')

In [23]:
df["bag_of_words"].isnull().sum()

0

In [24]:
# Convertir la colección de textos (en este caso, la bolsa de palabras)
# en una matriz TF-IDF.

# Vamos a eliminar las construcciones como a, an, the, and,
# pero que no tienen sentido para nosotros de nuestros DataFrames.
tfidf = TfidfVectorizer(stop_words="english")

In [25]:
# ajustar y transformar según el objeto tfidf
# Los de las filas son textos 'bolsa de palabras'. Los de las columnas son palabras únicas.

# Crear la matriz TF-IDF (Term Frequency-Inverse Document Frequency) a partir de la columna
# 'bag_of_words' del conjunto de datos. La matriz TF-IDF es una representación numérica
# de los documentos en función de la frecuencia de las palabras que contienen y de
# su importancia en el conjunto de documentos.
tfidf_matrix = tfidf.fit_transform(df['bag_of_words'])

In [26]:
# Cada fila representa un documento (en este caso, una película) y
# cada columna representa una palabra única en el conjunto de datos.

# Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada documento.
tfidf_matrix.shape

(20000, 26500)

In [27]:
#Si queremos ver todas las palabras únicas de las columnas

# Filas: Cada fila de la matriz corresponde a una película en el conjunto de datos.
# Por ejemplo, si tienes 10,000 películas, habrá 10,000 filas en la matriz.

# Columnas: Cada columna representa una palabra única en el conjunto de datos.
# La cantidad de columnas es igual al número total de palabras únicas en todas
# las bolsas de palabras de películas.

# Valores: Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada película.
# Estas puntuaciones miden la importancia relativa de una palabra en un documento en comparación
# con su frecuencia en el conjunto de documentos. Valores más altos indican que la palabra
# es más importante en el contexto de ese documento.

tfidf.get_feature_names_out()


array(['aa', 'aaliyah', 'aamir', ..., '独闯龙潭', '莫声版', '魔鬼司令'], dtype=object)

In [28]:
# tfidf scores
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Creación de la matriz de similitud del coseno

In [29]:
# calcula la similitud coseno entre todas las filas de la matriz TF-IDF.
# La similitud coseno mide el coseno del ángulo entre dos vectores y
# proporciona una medida de cuán similares son dos documentos.

# Calcula cos sim para todos los pares de documentos posibles uno por uno. En la matriz cosine_sim,
# cada película tiene similitudes entre sí

if IN_COLAB:
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    import cupy as cp
    # Convertir la matriz TF-IDF a cupy array
    tfidf_matrix_cupy = cp.array(tfidf_matrix.toarray())
    # Calcular la similitud del coseno utilizando cupy.matmul
    cosine_sim_cupy = cp.matmul(tfidf_matrix_cupy, tfidf_matrix_cupy.T)
    cosine_sim = cosine_sim_cupy
else:
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### OTRA FORMA DE HACER LA SMILITUD DEL COSENO

Calcula el producto punto (producto interno) entre pares de vectores o matrices de vectores. 
En el contexto de matrices, como la matriz TF-IDF, linear_kernel calcula el producto punto entre cada par de filas (documentos) en la matriz.

In [30]:
# from sklearn.metrics.pairwise import linear_kernel
# linear_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
# linear_similarity_matrix.shape


In [31]:
# cosine_sim[i, j] representa la similitud coseno entre la película con el índice i y
# la película con el índice j en el conjunto de datos.

cosine_sim.shape

(20000, 20000)

In [32]:
cosine_sim

array([[1.        , 0.00638517, 0.00800027, ..., 0.00150012, 0.00150587,
        0.0023972 ],
       [0.00638517, 1.        , 0.00898501, ..., 0.00993785, 0.01581025,
        0.01190703],
       [0.00800027, 0.00898501, 1.        , ..., 0.0036039 , 0.03747377,
        0.02370732],
       ...,
       [0.00150012, 0.00993785, 0.0036039 , ..., 1.        , 0.02796042,
        0.00649719],
       [0.00150587, 0.01581025, 0.03747377, ..., 0.02796042, 1.        ,
        0.01803002],
       [0.0023972 , 0.01190703, 0.02370732, ..., 0.00649719, 0.01803002,
        1.        ]])

In [33]:
cosine_sim[1].shape

(20000,)

# Recomendaciones basada en la similitud del coseno

In [108]:
# Vamos a crear una serie pd de índices y nombres de películas
# se crea una Serie de Pandas llamada indices donde los índices son
# los títulos de las películas y los valores son los índices originales del DataFrame df.
indices = pd.Series(df.index, index=df['title'])

In [109]:
indices.head()

title
Eyes Wide Shut (1999)                    0
Tokyo Story (Tôkyô monogatari) (1953)    1
Smokey and the Bandit (1977)             2
Picnic at Hanging Rock (1975)            3
Jacket, The (2005)                       4
dtype: int64

In [110]:
indices.shape

(20000,)

In [111]:
# contemos la información del índice de las películas y simplifiquemos las películas más repetitivas a las más recientes
indices.index.value_counts().head()

Star Wars: Episode IV - A New Hope (1977)    152
Pulp Fiction (1994)                           93
Forrest Gump (1994)                           78
Shawshank Redemption, The (1994)              74
Matrix, The (1999)                            69
Name: title, dtype: int64

### Se eliminan las películas duplicadas del índice

In [112]:
# Se eliminan las películas duplicadas del índice, conservando solo la última aparición de cada película.
# Esto se hace para evitar ambigüedades y asegurar que cada película tenga un índice único.
indices = indices[~indices.index.duplicated(keep='last')]

In [113]:
indices.index.value_counts().head()

Smokey and the Bandit (1977)         1
Once Upon a Time in Mexico (2003)    1
Heavy Metal (1981)                   1
Blow-Up (Blowup) (1966)              1
Italian Job, The (1969)              1
Name: title, dtype: int64

In [114]:
indices.shape

(5880,)

In [115]:
indices

title
Smokey and the Bandit (1977)                         2
Family Business (1989)                              14
Pirates of Silicon Valley (1999)                    17
Man with the Iron Fists, The (2012)                 20
I'm So Excited (Los amantes pasajeros) (2013)       41
                                                 ...  
Pelle the Conqueror (Pelle erobreren) (1987)     19995
Man in the Iron Mask, The (1998)                 19996
Cowspiracy: The Sustainability Secret (2014)     19997
Bolt (2008)                                      19998
10 Things I Hate About You (1999)                19999
Length: 5880, dtype: int64

### Pelicula a recomendar

In [116]:
movie_title = "Wolf of Wall Street, The (2013)"

In [117]:
# Asigno el índice de la película "Wolf of Wall Street, The (2013)" a la variable
movie_index = indices[movie_title]
movie_index

19548

### Para ver en qué se parece la película anterior a las demás películas

In [43]:
cosine_sim[movie_index]

array([0.06108063, 0.01546639, 0.00838258, ..., 0.03241003, 0.02223729,
       0.01322481])

In [44]:
cosine_sim[movie_index].shape

(20000,)

# Función de recomendación basada en contenido, dada una pelicula.

In [55]:
def content_based_recommender(title, cosine_sim, dataframe, num_recomendations=10):
    """
    Genera recomendaciones basadas en contenido para una película específica.
    Parameters:
    title (str): El título de la película para la cual se generarán las recomendaciones.
    cosine_sim (array-like): Matriz de similitud coseno entre películas.
    dataframe (DataFrame): DataFrame que contiene la información de las películas.
    num_recomendations (int): El número de recomendaciones a generar (por defecto es 10).
    Returns:
    Series: Una Serie de Pandas que contiene los títulos de las películas recomendadas.
    """
    # Crear índices basados en los títulos de las películas
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]  # Eliminar índices duplicados
    
    # Capturar el índice de la película seleccionada
    movie_index = indices[title]
    
    # Calcular los puntajes de similitud basados en el título
    if IN_COLAB:
        similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])
    else:
        similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    
    # Ordenar los índices de las películas por puntaje de similitud
    movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index
    
    # Eliminar títulos duplicados y excluir la película seleccionada
    duplicated_titles = dataframe.loc[movie_indices_sort, 'title'].duplicated(keep='first')
    indices_sin_duplicados = movie_indices_sort[~duplicated_titles]
    filas_sin_duplicados_y_exclusion = dataframe.loc[indices_sin_duplicados]
    filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != title]
    
    # Devolver los títulos de las películas recomendadas
    return filas_sin_duplicados_y_exclusion['title'].head(num_recomendations)


In [56]:
content_based_recommender("Wolf of Wall Street, The (2013)", cosine_sim, df)

4732                          Too Big to Fail (2011)
5249                              Wall Street (1987)
19567                             Margin Call (2011)
17695                          Big Short, The (2015)
18124                           Money Monster (2016)
15880                             Boiler Room (2000)
216                                Inside Job (2010)
12560    Enron: The Smartest Guys in the Room (2005)
12286                                 Traffic (2000)
2460                               Goodfellas (1990)
Name: title, dtype: object

In [62]:
content_based_recommender("Matrix, The (1999)", cosine_sim, df)

14358       Reservoir Dogs (1992)
18423         Jackie Brown (1997)
19474    Kill Bill: Vol. 2 (2004)
15212    Kill Bill: Vol. 1 (2003)
17529             Sin City (2005)
18467         True Romance (1993)
2388            Goodfellas (1990)
11012       Godfather, The (1972)
14437         Forrest Gump (1994)
14616     Django Unchained (2012)
Name: title, dtype: object

In [ ]:
content_based_recommender("Star Wars: Episode IV - A New Hope (1977)", cosine_sim, df)

In [ ]:
content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)

# Función de recomendación basada en contenido, dada una pelicula.

In [ ]:
# Para un usario se puede medir, pasandole el usario, y de las peliculas que ha visto ir sacando la recomendación por pelicula e ir evaluando el modelo porpelicula.

In [199]:
def content_based_recommender_for_user(user_id, cosine_sim, completeDataframe, df_bag_of_words, num_recommendations=10):
    """
    Genera recomendaciones basadas en contenido para un usuario específico.
    
    Parameters:
    user_id (int): El ID del usuario para el cual se generarán las recomendaciones.
    cosine_sim (array-like): Matriz de similitud coseno entre películas.
    dataframe (DataFrame): DataFrame que contiene la información de las películas.
    num_recomendations (int): El número de recomendaciones a generar (por defecto es 10).
    
    Returns:
    Series: Una Serie de Pandas que contiene los títulos de las películas recomendadas.
    """
    # Obtener las películas que el usuario ha visto o calificado previamente
    peliculas_vistas_usario = completeDataframe[completeDataframe['userId'] == user_id]
    peliculas_vistas_usario.dropna(subset=['title'])
    titulo_peliculas_vistas = peliculas_vistas_usario['title']
    print(f'\n Peliculas vistas por el usuario{user_id} --> \n{titulo_peliculas_vistas}\n')
    
    for titulo_pelicula in titulo_peliculas_vistas:
        print(f"\n Peliculas recomendadas según: {titulo_pelicula}: \n")
        peliculas_recomendadas = content_based_recommender(titulo_pelicula, cosine_sim, df_bag_of_words, num_recommendations)
        print(f"{peliculas_recomendadas}: \n\n")
        # En este punto hay que evaluar las recomendaciones (filas_sin_duplicados_y_exclusion) con las peliculas vistas por el usario (titulo_peliculas_vistas)

In [201]:
# Recomendar películas para el usuario con ID 123
num_recommendations = 10
dfCopy= dataFrame.copy()
recommended_movies = content_based_recommender_for_user(dfCopy['userId'].sample().iloc[0], cosine_sim, dataFrame,df, num_recommendations)
print(recommended_movies)


 Peliculas vistas por el usuario155146 --> 
22100557              Perfume: The Story of a Murderer (2006)
9239195                                      Airplane! (1980)
16660659                           Alien: Resurrection (1997)
23471588                              Love and Pigeons (1985)
8062830                       Resident Evil: Afterlife (2010)
8831810                                 Jurassic World (2015)
3679116     Master and Commander: The Far Side of the Worl...
24553599                       Sun Alley (Sonnenallee) (1999)
9567263                                       Zootopia (2016)
23471957                                   Bootleggers (1961)
8638549                      Grand Budapest Hotel, The (2014)
23672186                                        Infini (2015)
22895040                                 Atomic Blonde (2017)
11682860                          Escape from New York (1981)
21954759                                          Life (2017)
2959084                  

#### Creación de una columna nueva con los valores de similitud de una pelicula con el resto de peliculas. 

In [45]:
# # Veamos los Smilarity Scores que expresan las similitudes entre la película "Wolf of Wall Street, The (2013)" y otras películas
# if IN_COLAB:
#     similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])
# else:
#     similarity_scores = pd.DataFrame(cosine_sim[movie_index],
#                                     columns=["score"])

In [46]:
# # Las similitudes entre la película "Wolf of Wall Street, The (2013)" y el resto de películas
# similarity_scores.head()

In [47]:
# similarity_scores.shape

###  Ordena los índices de las películas en función de sus puntajes de similitud, de mayor a menor.

In [48]:
# movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index

In [49]:
# import pandas as pd

# # Supongamos que tu DataFrame se llama df y la Serie de índices es index_series
# # index_series = Int64Index([9859, 3228, 1656, 8951, 6811, 998, 5030, 6972, 8591, 4820, ...])

# # Paso 1
# duplicated_titles = df.loc[movie_indices_sort, 'title'].duplicated(keep='first')

# # Paso 2
# indices_sin_duplicados = movie_indices_sort[~duplicated_titles]

# # Paso 3
# filas_sin_duplicados = df.loc[indices_sin_duplicados]

# # Ahora filas_sin_duplicados contiene las filas correspondientes a los índices
# # en index_series, eliminando las filas con títulos duplicados, manteniendo la primera ocurrencia.

# # Supongamos que el string que deseas excluir es 'The Promise'
# string_a_excluir = movie_title

# # Filtrar las filas que no contienen el string a excluir en la columna 'title'
# filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
# filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]

In [50]:
# # Supongamos que el string que deseas excluir es 'The Promise'
# string_a_excluir = 'Wolf of Wall Street, The (2013)'

# # Filtrar las filas que no contienen el string a excluir en la columna 'title'
# filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
# filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]


In [51]:
# filas_sin_duplicados_y_exclusion.head()

In [52]:
# Vamos a listar las puntuaciones de similitud de la película 'Sherlock Holmes' en orden descendente. Empieza por 1 porque es la primera película.
# movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

In [53]:
# Go to the indexes we selected in our first data set
# df['title'].iloc[movie_indices]

In [60]:
# def calculate_cosine_sim(dataframe):
#     tfidf = TfidfVectorizer(stop_words='english')
#     dataframe['bag_of_words'] = dataframe['bag_of_words'].fillna('')
#     tfidf_matrix = tfidf.fit_transform(dataframe['bag_of_words'])
#     cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#     return cosine_sim

In [61]:
# cosine_sim = calculate_cosine_sim(df)
# content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)